In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Path to the txt file
file_path = './SOBRADINHO_DIARIA_1931_2016.txt'

# Reading file
with open(file_path, 'r') as file:
    lines = file.readlines()

# Creating a DataFrame
df = pd.DataFrame(lines, columns=['Flow'])

# Strip newline characters from the end of each line
df['Flow'] = df['Flow'].str.strip()
df['Flow']=pd.to_numeric(df['Flow'], errors='coerce').fillna(0).astype(int)

print(df)

       Flow
0      5512
1      5543
2      5607
3      5639
4      5703
...     ...
31407  2871
31408  3048
31409  3163
31410  3160
31411  3023

[31412 rows x 1 columns]


In [3]:
print(df.dtypes)

Flow    int64
dtype: object


In [ ]:
#função para fazer o deslocamentos das saídas e entradas
def deslocamento(entradas,saidas,colunas,tabela):
    #cria uma cópia da tabela
    tabela1=tabela.copy()
    #desloca a coluna flow i vezes para trás
    for i in range(1,entradas+1):
        for coluna in colunas:
            tabela1.insert(loc=0, column=coluna+str((entradas-i)+1), value=tabela[coluna].shift(i))
        
    #desloca a coluna flow i vezes para frente
    for i in range(1,saidas):
        tabela1['Saida'+ str(i)]=tabela['Flow'].shift(-1*i)
    return tabela1
    

In [ ]:
#armazenando os dados das tabelas de acordo com as entradas numa lista dataframes
entradas_testes=[15,30,45]
saidas = 12
dataframes = []
colunas = ['Flow']
for entrada in entradas_testes:
    dataframes.append(deslocamento(entrada,saidas,colunas,df).dropna().round(4))
    dataframes[-1].to_csv('arquivo1_'+str(entrada)+'.csv', index=False, sep = ' ')

In [ ]:
def calcularZeW(tabela):
    #cria uma cópia da tabela
    tabela1=tabela.copy()
    z=[]
    for i,vazao in enumerate(tabela['Flow']):
       z.append((tabela1.diff().iloc[i,0]/vazao)*100)
    tabela1.insert(loc=0, column='z', value=z)
    tabela1.insert(loc=0, column='w', value=tabela1['z'].diff())
    return tabela1
df_z_w=calcularZeW(df)   

In [ ]:
#armazenando os dados das tabelas de acordo com as entradas numa lista dataframes para o arquivo 2
entradas_testes=[15,30,45]
saidas = 12
dataframes2 = []
colunas = ['Flow','z','w']
for entrada in entradas_testes:
    df_res = deslocamento(entrada,saidas,colunas,df_z_w).dropna().round(4)
    df_res.to_csv('arquivo2_'+str(entrada)+'.csv', index=False,sep=' ')

In [ ]:
df_teste = pd.read_csv('arquivo2_15.csv',sep=' ')

df_teste

In [ ]:
def bits(valueZ,valueW):
    if valueZ>20 and valueW>0:
        return [1,0,0]
    elif valueZ<-20 and valueW<0:
        return [0,0,1]
    else:
        return [0,1,0]
def calcularDFbits(tabela):
    bits_0a2=tabela.apply(lambda x: bits(x.z,x.w),axis=1)
    df_bits=pd.DataFrame(np.array(bits_0a2.tolist()).reshape(-1,3),columns=['bit0','bit1','bit2'])
    df_bits=pd.concat([df_bits,tabela],axis=1)
    return df_bits.dropna().drop(['z','w'],axis=1)

In [ ]:
df_bits=calcularDFbits(df_z_w)

In [ ]:
df_bits

In [ ]:
#armazenando os dados das tabelas de acordo com as entradas numa lista dataframes para o arquivo 3
entradas_testes=[15,30,45]
saidas = 12
dataframes2 = []
colunas = ['Flow','bit2','bit1','bit0']
for entrada in entradas_testes:
    df_res = deslocamento(entrada,saidas,colunas,df_bits).dropna().round(4)
    df_res.to_csv('arquivo3_'+str(entrada)+'.csv', index=False,sep=' ')

In [ ]:
df_teste = pd.read_csv('arquivo3_15.csv',sep=' ')

df_teste